In [1]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

with beam.Pipeline(options=PipelineOptions()) as p:
    lines = p | beam.Create(['cat dog', 'snake cat', 'dog'])
    (lines
    | beam.FlatMap(lambda x: x.split(' '))    # One to many mapping, including none
    | beam.Map(lambda x: (x, 1))                # One to one mapping
    | beam.CombinePerKey(sum)                   # Like shuffle, previous phase should have yielded key,value pair
    | beam.ParDo(lambda k: print(k)))           # Generic map function. 


/opt/tljh/user/lib/python3.6/site-packages/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


('cat', 2)
('dog', 2)
('snake', 1)


In [7]:
# Class assignment - Write code
# From the household.csv file dump number of male and females.
import csv

with open('/home/data/household.csv','r') as f:
   with beam.Pipeline(options=PipelineOptions()) as p:
       data = csv.reader(f)
       print(next(data))
       lines = p | beam.Create(data)
       (lines
        | beam.Map(lambda x: ('m', 1) \
                   if x[2] == 'm' \
                   else ('f', 1))
        | beam.CombinePerKey(sum)
        | beam.ParDo(lambda k: print(k))
       )

['Household id', 'Age', 'Gender', 'Profession']
('m', 34)
('f', 40)


In [9]:
# Class assignment - Write code
# From the household.csv file dump number of children (<18 years), working-age (19-60 years), old (>60 years).

# THE GOAL IS TO COUNT EVERY OCCURANCE OF THE CATEGORY DESCRIBED ABOVE
with open('/home/data/household.csv','r') as f: # First we read data from the CSV file.
   with beam.Pipeline(options=PipelineOptions()) as p:
       data = csv.reader(f)
       print(next(data))
       lines = p | beam.Create(data)
       (lines
        | beam.Map(lambda x: ('children', 1) \
                   if int(x[1]) < 18 \
                   else ('working-age', 1) \
                   if 18 <= int(x[1]) <= 60 \
                   else ('old', 1))
        | beam.CombinePerKey(sum)
        | beam.ParDo(lambda k: print(k))
       )

['Household id', 'Age', 'Gender', 'Profession']
('working-age', 48)
('children', 21)
('old', 5)


In [11]:
# Class assignment - Describe phases and transforms, no need to code.
# From the household.csv how would you compute median savings using beam?
import pandas as pd

data1 = pd.read_csv(r'/home/data/household.csv')
data1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 4 columns):
Household id    74 non-null int64
Age             74 non-null int64
Gender          74 non-null object
Profession      74 non-null object
dtypes: int64(2), object(2)
memory usage: 2.4+ KB


In [12]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

def output2(num):
   if num < 5:
       return []
   if num <= 10:
       return [("cube", num * num * num)]
   else :
       return [("square", num * num)]

def mySum(input):
#    ('cube', [1000, 343])
   (k, v) = input
   sumx = 0
   for vi in v:
       sumx += vi
   return (k, sumx)

with beam.Pipeline(options=PipelineOptions()) as p:
   lines1 = p | "create1" >> beam.Create([1, 3, 10])
   lines2 = p | "create2" >> beam.Create([15, 12, 7])
   lines = (lines1, lines2) | beam.Flatten()
   lines = lines | beam.FlatMap(output2)
   (lines | "l1" >> beam.ParDo(lambda k: print("original", k)))           # Generic map function.
   linesGrouped = lines | beam.GroupByKey()
   linesGrouped = linesGrouped | beam.Map(mySum)
   (linesGrouped | "lg" >> beam.ParDo(lambda k: print(k)))           # Generic map function.

original ('cube', 1000)
original ('square', 225)
original ('square', 144)
original ('cube', 343)
('cube', 1343)
('square', 369)
